The following were generated using the prompt from `prompts.md` and `Mistral (7B) Instruct vo-2` on [together.ai](https://api.together.xyz/playground/chat/mistralai/Mistral-7B-Instruct-v0.2)

In [1]:
influencer_fit = "This Instagram profile, belonging to Sabrina Bloedorn (@sabrina.bloedorn), showcases a passionate adventurer and fashion enthusiast who combines her love for the outdoors with stylish outfits. Sabrina's feed primarily features her outdoor adventures, including hiking roundups, and her unique take on outfits using various brands such as Peak Performance, Oak and Fort, VivoBarefoot, and HikeClerb. She is also known for her creative upcycling projects, like transforming an Arc'teryx hoody into a baby onesie. Sabrina has collaborated with various brands and has been featured on Hypebae. Her feed exudes a sense of adventure, self-expression, and sustainability."
brand = "This Instagram profile, under the handle @spaceout_official, represents the brand Space Out, specializing in clothing. The profile showcases a connection to the outdoors, encouraging followers to 'enjoy the outside moment' with the hashtag #getoutside. The brand highlights its products, such as the Toray Delfy 3L rain jacket, with a focus on their innovative details and functionality. The profile also features content that goes beyond traditional outdoor activities, showcasing city hikes and urban exploration. #spaceout #notjustforoutdoors."
influencer_nofit = "This Instagram profile, @reecebowden, belongs to Reece Bowden, a Fitness Trainer. Reece shares a mix of fitness-related content, including workout routines, training tips, and motivation. He encourages his followers to build their strength day by day and explores various forms of training, from running to hybrid workouts. His posts often feature inspiring messages and the use of the hashtag #explore. Reece also offers online coaching and collaborations, and can be reached at reecebowdencollabs@gmail.com."

In [2]:
import together
from together import Together
import numpy as np

TOGETHER_API_KEY = "5a532872525382e32ebc396c6cc682d3b8d8d5ea428ef9468404286bb1417f2c"

def generate_embeddings(input_texts: list,
                        model_api_string: str
                        ) -> list:
    """Generate embeddings from Together python library.

    Args:
        input_texts: a list of string input texts.
        model_api_string: str. An API string for a specific embedding model of your choice.

    Returns:
        embeddings_list: a list of embeddings. Each element corresponds to the each input text.
    """
    together_client = together.Together(api_key = TOGETHER_API_KEY)
    outputs = together_client.embeddings.create(
        input=input_texts,
        model=model_api_string,
    )
    return np.array([x.embedding for x in outputs.data])

embeddings = generate_embeddings([influencer_fit, influencer_nofit], 'BAAI/bge-base-en-v1.5')
np.save('embeddings.npy', embeddings)

In [37]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-base-en-v1.5")

sentences = [
    # brand,
    influencer_fit,
    influencer_nofit
]
embeddings = model.encode(sentences)

In [43]:
import pandas as pd

df = pd.DataFrame({'description': sentences})
df.to_csv('description.csv', index=False)

In [27]:
import numpy as np

def cosine_similarity(vectors, target_vector):
    """
    Compute the cosine similarity between a set of vectors and a target vector.

    Parameters:
        vectors (array-like): A 2D array where each row is a vector.
        target_vector (array-like): A 1D array representing the target vector.

    Returns:
        np.ndarray: A 1D array containing cosine similarities for each vector in the set.
    """
    # Convert to numpy arrays if not already
    vectors = np.array(vectors)
    target_vector = np.array(target_vector)
    
    # Compute dot products between each vector and the target vector
    dot_products = np.dot(vectors, target_vector)
    
    # Compute norms of the vectors
    vector_norms = np.linalg.norm(vectors, axis=1)
    target_norm = np.linalg.norm(target_vector)
    
    # Avoid division by zero
    if target_norm == 0:
        return np.zeros_like(dot_products)
    
    # Compute cosine similarities
    return dot_products / (vector_norms * target_norm)

In [ ]:
similarities = model.similarity(embeddings[2], embeddings[1])
print(similarities)

tensor([[0.6422]])


## Together.ai

[API Docs](https://docs.together.ai/reference/chat-completions-1)

In [175]:
from together import Together
import numpy as np

TOGETHER_API_KEY = "5a532872525382e32ebc396c6cc682d3b8d8d5ea428ef9468404286bb1417f2c"
client = Together(api_key = TOGETHER_API_KEY)

def cosine_similarity(vectors,
                      target_vector
                      ):
    """
    Compute the cosine similarity between a set of vectors and a target vector.

    Parameters:
        vectors (array-like): A 2D array where each row is a vector.
        target_vector (array-like): A 1D array representing the target vector.

    Returns:
        np.ndarray: A 1D array containing cosine similarities for each vector in the set.
    """
    if not isinstance(vectors, np.ndarray):
        vectors = np.array(vectors)
    if not isinstance(target_vector, np.ndarray):
        target_vector = np.array(target_vector)
    
    dot_products = np.dot(vectors, target_vector)

    vector_norms = np.linalg.norm(vectors, axis=1)
    target_norm = np.linalg.norm(target_vector)
    
    if target_norm == 0:
        return np.zeros_like(dot_products)
    
    return dot_products / (vector_norms * target_norm)

def retrieve(client,
             query,
             vdb = None,
             descriptions = None,
             ):
    """Retrieve the most appropriate influencer's description from the influencer database using Together API to embed the query.

    Args:
        query: str. The input query.
        vdb: np.ndarray. A set of vectors representing influencer descriptions.
        descriptions: list. A list of corresponding descriptions.

    Returns:
        str: The retrieved text.
    """
    
    if vdb is None:
        vdb = np.load('embeddings.npy')
        
    if descriptions is None:
        descriptions = open('descriptions.txt', 'r').read().replace("\"","").splitlines()

    model = "BAAI/bge-base-en-v1.5"

    out = client.embeddings.create(
        input=query,
        model=model,
    )

    query = out.data[0].embedding

    similarities = cosine_similarity(vdb, query)

    idx = np.argmax(similarities)
    
    return descriptions[idx]

def generate(client,
             TASK_TYPE=None,        
             userPrompt=None,
             imageUrl=None
             ):
    """Generate text using the Together API.

    Args:
        TASK_TYPE: str. The type of task to generate text for. Options are "summarize_post", "summarize_influencer", "summarize_brand", "rag".
        userPrompt: str. The user prompt to provide to the model.
        imageUrl: str. The URL of the image to be described (required for "summarize_post" task).

    Returns:
        str: The generated text.
    """

    if TASK_TYPE is None:
        raise ValueError("TASK_TYPE cannot be None.")
    
    if TASK_TYPE == "summarize_post":
        
        model = "meta-llama/Llama-Vision-Free"
        systemInstruction = open('./prompts/describe_image.txt', 'r').read()
        userPrompt = "Summarize the content of this instagram post"
        
        messages=[
            {
                "role": "system",
                "content": f"{systemInstruction}",
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": userPrompt}, {"type": "image_url", "image_url": {"url": imageUrl}}],
            }]

    else:
        
        model = "meta-llama/Llama-3.2-3B-Instruct-Turbo"

        if TASK_TYPE == "summarize_influencer":
            systemInstruction = open('./prompts/summarize_influencer.txt', 'r').read()
            userPrompt = "Summarize the content of this influencer's profile:\n\n" + userPrompt
        
        elif TASK_TYPE == "summarize_brand":
            systemInstruction = open('./prompts/summarize_brand.txt', 'r').read()
            userPrompt = "Summarize the content of this brand's profile:\n\n" + userPrompt
        
        elif TASK_TYPE == "rag":
            systemInstruction = open('./prompts/rag.txt', 'r').read().format(retrieve(client, userPrompt))
            userPrompt = "Explain why the provided influencer is a strong match for the following brand:\n\n" + userPrompt
        
        else:
            raise ValueError(f"Invalid TASK_TYPE: {TASK_TYPE}.")
        
        messages=[
            {
                "role": "system",
                "content": f"{systemInstruction}",
            },
            {
                "role": "user",
                "content": userPrompt,
            }]
        
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=512,
        temperature=0.7,
        top_p = 0.7,
        top_k = 50,
        stream=False,
    )
    return stream.choices[0].message.content.replace('. ', '.\n')

def generate_embeddings(client,
                        queries: list,
                        vdb = None):
    """Generate embeddings using Together API and store the original description in a text file and the embeddings in a numpy file.

    """

    if vdb is None:
        vdb = np.load('embeddings.npy')

    # Make every description a single line
    queries = [x.replace('\n', ' ') for x in queries]

    model = "BAAI/bge-base-en-v1.5"

    out = client.embeddings.create(
        input=queries,
        model=model,
    )

    vdb_new = np.array([x.embedding for x in out.data])

    vdb = np.vstack((vdb, vdb_new))
    np.save('embeddings.npy', vdb)

    with open('descriptions.txt', 'a') as f:
        for query in queries:
            f.write(query + '\n')

    return f"Added {len(queries)} new descriptions to the database."

In [ ]:
query = "I love going to the mountains"
out = retrieve(client, query)
print

This Instagram profile, belonging to Sabrina Bloedorn (@sabrina.bloedorn), showcases a passionate adventurer and fashion enthusiast who combines her love for the outdoors with stylish outfits. Sabrina's feed primarily features her outdoor adventures, including hiking roundups, and her unique take on outfits using various brands such as Peak Performance, Oak and Fort, VivoBarefoot, and HikeClerb. She is also known for her creative upcycling projects, like transforming an Arc'teryx hoody into a baby onesie. Sabrina has collaborated with various brands and has been featured on Hypebae. Her feed exudes a sense of adventure, self-expression, and sustainability.


In [ ]:
userPrompt = """Write a summary of this Instagram profile:

Instagram Handle:
@

Bio:

Post 1:

Post 2:

Post 3:

Post 4:

Post 5:

"""

out = generate(client=client, 
               TASK_TYPE='summarize_influencer',
               userPrompt=userPrompt
               )

print(out)

This profile showcases Jeremy Richardson, a digital creator and talent agent, primarily featuring his experiences as a cowboy and rancher in Montana.
The content primarily revolves around his love for ranching, cowboy culture, and the outdoors, often incorporating elements of his work on the TV series "Yellowstone." Jeremy frequently shares photographs and videos of his daily life on the range, showcasing his skills as a rancher and cowboy.
His posts often include hashtags related to ranching, cowboy culture, and his personal journey as a sober cowboy.
Notably, Jeremy has been featured on "Yellowstone" and will appear in the second part of season 5, set to air on Paramount.
His engaging posts often express gratitude and appreciation for his life and friendships, highlighting his down-to-earth and authentic personality.
